In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [6]:
import pandas as pd
import numpy as np
import string
import pickle
import io
import torch
from sentence_transformers import SentenceTransformer

metadata = pd.read_csv("Movies_dataset.csv")

with open('encodings_description.pickle','rb') as file:
    encoded_data = pickle.load(file)

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

with open('model.pickle','rb') as file:
    #model = pickle.load(file)
    model = CPU_Unpickler(file).load()

def find_similar_items(query: str, metadata: pd.DataFrame, model: SentenceTransformer, encodings: np.array, top_n: int = 5):
    query_vector = model.encode([query])
    # print(query_vector)
    similarity = np.dot(encodings,query_vector.T)
    # print(similarity)
    top_items = similarity.flatten().argsort()[-top_n:][::-1]
    # print(top_items)
    # return metadata['asin'].iloc[top_items]
    return list(metadata['movieId'].iloc[top_items].values)

In [10]:
top_n = 10
query = "superheros assemble to fight a space tyrant"

top_items_mID = find_similar_items(query=query, metadata=metadata, model=model, encodings=encoded_data, top_n=top_n)

In [11]:
for i in range(10):
    print(metadata[metadata['movieId']==top_items_mID[i]]['title'])

23093    Attack from Space
Name: title, dtype: object
26132    JLA Adventures: Trapped in Time
Name: title, dtype: object
6946    Royal Space Force - The Wings Of Honneamise
Name: title, dtype: object
22407    Justice League: War
Name: title, dtype: object
25735    V: The Final Battle
Name: title, dtype: object
23144    Atomic Rulers
Name: title, dtype: object
8120    Sky Captain and the World of Tomorrow
Name: title, dtype: object
22408    Gamera vs. Viras
Name: title, dtype: object
23579    Valley of the Dragons
Name: title, dtype: object
5279    Hero and the Terror
Name: title, dtype: object


In [5]:
metadata.iloc[[983,16789,39439,17378],:]

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url
983,1039,1023,63819,81310.0,Winnie the Pooh and the Blustery Day,Winnie the Pooh and his friends experience hig...,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'name': 'Walt Disney Productions', 'id': 3166}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1968-12-20,25.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 193143, 'name': 'winnie the pooh'}, {'...","[{'cast_id': 2, 'character': 'Winnie the Pooh'...","[{'credit_id': '52fe48079251416c9107dc53', 'de...",NF
16789,17284,85585,170811,24926.0,Winnie the Pooh and a Day for Eeyore,Winnie the Pooh and friends decide to throw a ...,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'name': 'Walt Disney Productions', 'id': 3166}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1983-01-01,25.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 4809, 'name': 'tiger'}, {'id': 4931, '...","[{'cast_id': 1004, 'character': 'Narrator', 'c...","[{'credit_id': '52fe44b0c3a368484e0312df', 'de...",NF
39439,41681,163072,211729,36161.0,Winnie-the-Pooh,"With a cheeky, down-to-earth charm that appeal...",0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'name': 'Soyuzmultfilm', 'id': 14599}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1969-01-01,11.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]","[{'id': 9903, 'name': 'balloon'}, {'id': 15101...","[{'cast_id': 1001, 'character': 'Winnie-the-Po...","[{'credit_id': '52fe45d19251416c9103f64b', 'de...",NF
17378,17909,88267,1449283,51162.0,Winnie the Pooh,"During an ordinary day in Hundred Acre Wood, W...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2011-04-13,63.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 3905, 'name': 'owl'}, {'id': 4809, 'na...","[{'cast_id': 1004, 'character': 'Winnie the Po...","[{'credit_id': '5542d8a9925141586100090e', 'de...",NF
